# ObiTools

In [1]:
cd ../Desktop/School/FISH546_Git/Sam-Metabarcoding/Data/Bat_Data

/Users/tracykreling/Desktop/School/FISH546_Git/Sam-Metabarcoding/Data/Bat_Data


In [38]:
!head ngs_filter_bat.txt

#exp	sample	forward_primer	reverse_primer	extra
bat	1_S1_L001	AGATATTGGAACWTTATATTTTATTTTTGG	WACTAATCAATTWCCAAATCCTCC	
bat	2_S2_L001	AGATATTGGAACWTTATATTTTATTTTTGG	WACTAATCAATTWCCAAATCCTCC	

### Import the data

In [15]:
!obi import --fastq-input 2_S2_L001_R1_001.fastq bat/reads1

2021-02-28 10:52:49,246 [import : INFO ]  obi import: imports an object (file(s), obiview, taxonomy...) into a DMS
2021-02-28 10:52:49,259 [import : INFO ]  Opened file: 2_S2_L001_R1_001.fastq
2021-02-28 10:52:49,404 [import : INFO ]  Importing 64274 entries
 100.0 % |##################################################/] remain : 00:00:00
2021-02-28 10:52:51,932 [import : INFO ]  Imported 64274 entries
2021-02-28 10:52:52,085 [import : INFO ]  Done.


In [16]:
!obi import --fastq-input 2_S2_L001_R2_001.fastq bat/reads2

2021-02-28 10:52:52,442 [import : INFO ]  obi import: imports an object (file(s), obiview, taxonomy...) into a DMS
2021-02-28 10:52:52,455 [import : INFO ]  Opened file: 2_S2_L001_R2_001.fastq
2021-02-28 10:52:52,588 [import : INFO ]  Importing 64274 entries
 100.0 % |##################################################/] remain : 00:00:00
2021-02-28 10:52:55,168 [import : INFO ]  Imported 64274 entries
2021-02-28 10:52:55,297 [import : INFO ]  Done.


In [30]:
!obi import --ngsfilter ngs_filter_bat.txt bat/ngsfile

2021-02-28 16:19:28,466 [import : INFO ]  obi import: imports an object (file(s), obiview, taxonomy...) into a DMS
2021-02-28 16:19:28,482 [import : INFO ]  Opened file: ngs_filter_bat.txt
2021-02-28 16:19:28,515 [import : INFO ]  Importing 2 entries
 100.0 % |##################################################\] remain : 00:00:00
2021-02-28 16:19:29,576 [import : INFO ]  Imported 2 entries
2021-02-28 16:19:29,706 [import : INFO ]  Done.


### Check what is in the file structure

In [31]:
!obi ls bat

2021-02-28 16:19:34,829 [ls : INFO ]  obi ls
# DMS name: bat
# Views:
  # aligned_reads: Date created: Sun Feb 28 10:53:21 2021 ; Line count: 64274
  # reads1: Date created: Sun Feb 28 10:52:49 2021 ; Line count: 64274
  # ngsfile: Date created: Sun Feb 28 16:19:29 2021 ; Line count: 2
  # good_sequences: Date created: Sun Feb 28 10:53:39 2021 ; Line count: 64267
  # reads2: Date created: Sun Feb 28 10:52:52 2021 ; Line count: 64274
# Taxonomies:



### Recover the full sequeunces from the partial foward and reverse reads

In [33]:
! obi alignpairedend -R bat/reads2 bat/reads1 bat/aligned_reads

2021-02-28 16:20:07,611 [alignpairedend : INFO ]  obi alignpairedend
 100.0 % |##################################################-] remain : 00:00:00
2021-02-28 16:20:16,427 [alignpairedend : INFO ]  Done.


### Remove unaligned sequence records

In [34]:
!obi grep -a mode:alignment bat/aligned_reads bat/good_sequences

2021-02-28 16:20:17,633 [grep : INFO ]  obi grep
 100.0 % |##################################################/] remain : 00:00:00
2021-02-28 16:20:18,334 [grep : INFO ]  Grepped 64267 entries
2021-02-28 16:20:18,386 [grep : INFO ]  Done.


### Export aligned reads

In [35]:
!obi alignpairedend -R bat/reads2 bat/reads1 - > aligned_reads.fastq

2021-02-28 16:21:02,857 [alignpairedend : INFO ]  obi alignpairedend
 100.0 % |##################################################\] remain : 00:00:00
2021-02-28 16:21:10,252 [alignpairedend : INFO ]  Printing to output...
 100.0 % |##################################################/] remain : 00:00:00
2021-02-28 16:21:13,054 [alignpairedend : INFO ]  Done.


### Assign each sequence record to the corresponding sample/marker combo

In [36]:
!obi ngsfilter -t bat/ngsfile -u bat/unidentified_sequences bat/good_sequences bat/identified_sequences

2021-02-28 16:21:38,548 [ngsfilter : INFO ]  obi ngsfilter
Traceback (most recent call last):
  File "/Users/tracykreling/obitools3/obi3-env/bin/obi", line 4, in <module>
    __import__('pkg_resources').run_script('OBITools3==3.0.0b40', 'obi')
  File "/Users/tracykreling/obitools3/obi3-env/lib/python3.9/site-packages/pkg_resources/__init__.py", line 665, in run_script
    self.require(requires)[0].run_script(script_name, ns)
  File "/Users/tracykreling/obitools3/obi3-env/lib/python3.9/site-packages/pkg_resources/__init__.py", line 1463, in run_script
    exec(code, namespace, namespace)
  File "/Users/tracykreling/obitools3/obi3-env/lib/python3.9/site-packages/OBITools3-3.0.0b40-py3.9-macosx-10.9-x86_64.egg/EGG-INFO/scripts/obi", line 62, in <module>
    config[root_config_name]['module'].run(config)
  File "python/obitools3/commands/ngsfilter.pyx", line 622, in obitools3.commands.ngsfilter.run
  File "python/obitools3/commands/ngsfilter.pyx", line 179, in obitools3.commands.ngsfilter.

### Dereplicate

Something going wrong here for some reason. I skipped the ngs filter step and maybe that is it, but it says "Unlike the OBITools1, the OBITools3 make it possible to run ngsfilter before aligning the paired-end reads, BUT it is not recommended to do so for usual data, as ngsfilter will not be able to detect and cut out partially sequenced primers." And I also don't know which Illumina tags are on this sample and this command can't handle partials either so not sure what to do here

In [25]:
!obi uniq -m sample bat/good_sequences bat/dereplicated_sequences

2021-02-28 16:05:34,674 [uniq : INFO ]  obi uniq
2021-02-28 16:05:35,589 [uniq : INFO ]  First browsing through the input
  51.0 % |#########################|                         ] remain : 00:00:00Traceback (most recent call last):
  File "python/obitools3/commands/uniq.pyx", line 636, in obitools3.commands.uniq.run
  File "python/obitools3/commands/uniq.pyx", line 357, in obitools3.commands.uniq.uniq_sequences
KeyError: b'MERGED_sample'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tracykreling/obitools3/obi3-env/bin/obi", line 4, in <module>
    __import__('pkg_resources').run_script('OBITools3==3.0.0b40', 'obi')
  File "/Users/tracykreling/obitools3/obi3-env/lib/python3.9/site-packages/pkg_resources/__init__.py", line 665, in run_script
    self.require(requires)[0].run_script(script_name, ns)
  File "/Users/tracykreling/obitools3/obi3-env/lib/python3.9/site-packages/pkg_resources/__init__.py", line 1463,

In [14]:
!obi clean_dms bat

2021-02-12 15:47:16,785 [clean_dms : INFO ]  obi clean_dms
2021-02-12 15:47:16,789 [clean_dms : INFO ]  Done.
